<a href="https://colab.research.google.com/github/misrori/trading_strategy/blob/main/rsi_base_strategy_stock_tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install goldhand
# restart session
import os
os.kill(os.getpid(), 9)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 6.8 MB/s eta 0:00:00
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=47f969166e5b266ae78fd7a99c40da2bc7b0b0b89519da468469c8657931312c
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires n

In [150]:
from goldhand import *
import pandas as pd
import numpy as np

tw = Tw()
stocks = tw.stock
stocks.head()



,logoid,name,close,change,change_abs,Recommend.All,volume,Value.Traded,market_cap_basic,price_earnings_ttm,...,update_mode,pricescale,minmov,fractional,minmove2,Mom[1],RSI7[1],Rec.Stoch.RSI,currency,fundamental_currency_code
0,apple,AAPL,190.42,-4.239376,-8.43,-0.490909,121879321.0,2.320826e+10,2.860503e+12,30.271525,...,delayed_streaming_900,100,1,false,0,-22.68,45.799004,0.0,USD,USD
1,microsoft,MSFT,381.35,-2.340649,-9.14,-0.178788,38024357.0,1.450059e+10,2.834949e+12,30.715390,...,delayed_streaming_900,100,1,false,0,0.52,62.407644,0.0,USD,USD
2,nvidia,NVDA,107.57,-5.912709,-6.76,-0.312121,437800087.0,4.709416e+10,2.624708e+12,36.594659,...,delayed_streaming_900,100,1,false,0,0.57,60.699941,0.0,USD,USD
3,amazon,AMZN,181.22,-5.170068,-9.88,-0.536364,68294063.0,1.237625e+10,1.920520e+12,32.817820,...,delayed_streaming_900,100,1,false,0,-10.03,54.623099,0.0,USD,USD
4,alphabet,GOOG,155.37,-3.532845,-5.69,-0.378788,35270504.0,5.479978e+09,1.876893e+12,19.310216,...,delayed_streaming_900,100,1,false,0,-6.08,56.653862,0.0,USD,USD


In [151]:
top_n = 10
top_stocks = stocks.groupby('sector').apply(lambda x: x.nlargest(top_n, 'market_cap_basic'))
top_stocks = top_stocks.reset_index( drop=True)  # Reset index
top_stocks.head()

,logoid,name,close,change,change_abs,Recommend.All,volume,Value.Traded,market_cap_basic,price_earnings_ttm,...,update_mode,pricescale,minmov,fractional,minmove2,Mom[1],RSI7[1],Rec.Stoch.RSI,currency,fundamental_currency_code
0,visa,V,324.61,-2.352375,-7.82,-0.403030,8709762.0,2.827276e+09,6.341505e+11,33.158997,...,delayed_streaming_900,100,1,false,0,-11.75,50.779632,0.0,USD,USD
1,s-and-p-global,SPGI,457.76,-3.424123,-16.23,-0.490909,1685316.0,7.714703e+08,1.436637e+11,37.065587,...,delayed_streaming_900,100,1,false,0,-33.81,46.259108,0.0,USD,USD
2,moodys,MCO,423.03,-3.547733,-15.56,-0.536364,1330598.0,5.628829e+08,7.609858e+10,37.560932,...,delayed_streaming_900,100,1,false,0,-31.51,49.716461,0.0,USD,USD
3,paypal,PYPL,60.56,-5.301016,-3.39,-0.536364,14398120.0,8.719501e+08,5.990852e+10,15.149089,...,delayed_streaming_900,100,1,false,0,-6.02,50.389128,0.0,USD,USD
4,equifax,EFX,217.73,-2.989663,-6.71,-0.490909,1376397.0,2.996829e+08,2.717242e+10,45.045101,...,delayed_streaming_900,100,1,false,0,-17.29,45.508002,0.0,USD,USD


In [152]:

def rsi_strategy(data, buy_threshold = 30, sell_threshold = 70):
    """
    RSI strategy for backtesting with Backtest class

    Parameters:
    - data: pandas DataFrame with columns: date, open, high, low, close, volume and rsi
    - buy_threshold: int, default 30,  buy when RSI is below this value
    - sell_threshold: int, default 70, sell when RSI is above this value
    """

    in_trade = False  # Flag to track if already in a trade
    trade_id = 1
    all_trades = []

    temp_trade = {}

    for i in range(1, len(data)):
        # Check if not already in a trade
        if not in_trade:
            # Generate buy signal
            #You have to change olne the buy and sell signal
            if data['rsi'][i] < buy_threshold:

                if i == (len(data) -1):  # this is if the last wor have the signal there is no i+1
                    temp_trade['buy_price'] = data['close'][i]
                    temp_trade.update(dict(data.iloc[i].add_prefix('buy_')))
                else:
                    temp_trade['buy_price'] = data['open'][i+1]
                    temp_trade.update(dict(data.iloc[i+1].add_prefix('buy_')))

                temp_trade['trade_id'] = trade_id
                temp_trade['status'] = 'open'
                in_trade = True  # Set flag to indicate in a trade
        else:
            # Generate sell signal
            #You have to change olne the buy and sell signal
            if data['rsi'][i] > sell_threshold:
                if i == (len(data) -1):  # this is if the last wor have the signal there is no i+1
                    temp_trade['sell_price'] = data['close'][i]
                    temp_trade.update(dict(data.iloc[i].add_prefix('sell_')))
                else:
                    temp_trade['sell_price'] = data['open'][i+1]
                    temp_trade.update(dict(data.iloc[i+1].add_prefix('sell_')))

                temp_trade['trade_id'] = trade_id
                temp_trade['status'] = 'closed'


                # calculate results
                temp_trade['result'] = temp_trade['sell_price'] / temp_trade['buy_price']
                temp_trade['days_in_trade'] = (temp_trade['sell_date'] - temp_trade['buy_date']).days

                in_trade = False  # Reset flag to indicate not in a trade
                trade_id +=1
                all_trades.append(temp_trade)
                temp_trade = {}

    if temp_trade:
        temp_trade['sell_price'] = data['close'][i]
        temp_trade['trade_id'] = trade_id
        temp_trade['sell_date'] = data['date'][i]

        temp_trade['result'] = temp_trade['sell_price'] / temp_trade['buy_price']
        temp_trade['days_in_trade'] = (temp_trade['sell_date'] - temp_trade['buy_date']).days
        all_trades.append(temp_trade)

    res_df = pd.DataFrame(all_trades)

    # change orders
    all_col = res_df.columns.tolist()
    first = ['result', 'buy_price', 'sell_price', 'buy_date', 'sell_date', 'days_in_trade']
    first.extend([x for x in all_col if x not in first])
    res_df = res_df[first]
    return(res_df)


In [166]:
from IPython.display import Javascript
from goldhand import *
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 8000})'''))

symbol = 'TSLA'
data=GoldHand(symbol).df


backtest = Backtest( data, rsi_strategy, buy_threshold=25, sell_threshold=80)

backtest.trades_summary



<IPython.core.display.Javascript object>

{'ticker': 'TSLA',
 'number_of_trades': 7,
 'win_ratio(%)': 100.0,
 'average_res(%)': 72.9,
 'average_trade_len(days)': 337.0,
 'median_res(%)': 27.7,
 'cumulative_result': '21.84 x',
 'trade_results': '132.91 # 29.49 # 27.7 # 274.97 # 6.75 # 26.45 # 12.02',
 'profitable_trade_results': '132.91 # 29.49 # 27.7 # 274.97 # 6.75 # 26.45 # 12.02',
 'profitable_trades_mean': 72.9,
 'profitable_trades_median': 27.7,
 'looser_trade_results': '',
 'looser_trades_mean': nan,
 'looser_trades_median': nan,
 'median_trade_len(days)': 234.0,
 'number_of_win_trades': 7,
 'number_of_lost_trades': 0,
 'max_gain(%)': 274.97,
 'max_lost(%)': 6.75,
 'first_trade_buy': datetime.date(2012, 1, 17),
 'first_data_date': '2010-06-29',
 'first_open_price': 1.27,
 'last_data_date': '2025-04-10',
 'last_close_price': 252.4,
 'hold_result': '199.26 x',
 'buy_threshold': 25,
 'sell_threshold': 80}

In [167]:
backtest.trades


,ticker,result,buy_price,sell_price,buy_date,sell_date,days_in_trade,buy_open,buy_low,buy_high,...,sell_sma_100,sell_diff_sma100,sell_sma_200,sell_diff_sma200,sell_bb_lower,sell_bb_upper,sell_diff_upper_bb,sell_diff_lower_bb,sell_local,sell_local_text
0,TSLA,2.329075,1.774667,4.133333,2012-01-17,2013-05-07,476,1.774667,1.760667,1.822667,...,2.618180,41.345016,2.306583,60.439334,3.402363,3.982970,7.628442,-8.060804,,
1,TSLA,1.294934,12.870667,16.666668,2014-12-17,2017-01-24,769,12.870667,12.843333,13.776667,...,13.601720,24.793047,14.279070,18.873294,15.680563,17.129571,0.916522,-7.620113,,
2,TSLA,1.277009,17.099333,21.836000,2018-03-29,2019-10-28,578,17.099333,16.547333,18.063999,...,15.806087,38.221018,16.885543,29.384844,15.181438,23.906029,9.423095,-30.511254,,
3,TSLA,3.749693,24.980000,93.667336,2020-03-19,2020-07-07,110,24.980000,23.897333,30.133333,...,51.634480,79.448565,39.082790,137.079629,65.319567,99.201500,7.062761,-29.504159,,
4,TSLA,1.067511,210.039993,224.220001,2022-10-17,2023-06-08,234,210.039993,209.449997,221.860001,...,183.123000,28.252596,196.865450,19.299755,208.177304,236.750699,0.805032,-11.361107,,
5,TSLA,1.264474,185.500000,234.559998,2024-01-26,2024-07-03,159,185.500000,182.100006,186.779999,...,179.570300,37.210886,205.556900,19.864621,177.924292,255.199707,3.575514,-27.787535,,
6,TSLA,1.120234,225.309998,252.399994,2025-03-11,2025-04-10,30,225.309998,217.020004,237.059998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
backtest.show_trades()


In [154]:
# apply the backtest and save hte summary and at the aend create a df with a for loop ant tqdm


from tqdm import tqdm
import pandas as pd
summary_list = []
for i in tqdm(top_stocks.name.unique()):
    data=GoldHand(i).df
    backtest = Backtest(data, rsi_strategy, buy_threshold=30, sell_threshold=80)
    summary_list.append(backtest.trades_summary)
summary_df = pd.DataFrame(summary_list)
summary_df = summary_df.merge(top_stocks[['name', 'sector', 'industry', 'description']], left_on='ticker', right_on='name', how='left')
summary_df['cumulative_result(%)'] = ((summary_df['cumulative_result'].str.replace(' x', '').astype(float)) - 1) *100
summary_df['hold_result(%)'] = ((summary_df['hold_result'].str.replace(' x', '').astype(float)) - 1) *100

summary_df.reset_index(drop=True, inplace=True)
df = summary_df



100%|██████████| 200/200 [01:21<00:00,  2.46it/s]


In [156]:
summary_df.iloc[0]


,0
ticker,V
number_of_trades,5
win_ratio(%),100.0
average_res(%),118.09
average_trade_len(days),998.0
median_res(%),53.02
cumulative_result,21.37 x
trade_results,409.1 # 71.64 # 50.47 # 53.02 # 6.21
profitable_trade_results,409.1 # 71.64 # 50.47 # 53.02 # 6.21
profitable_trades_mean,118.09


In [163]:
summary_df = summary_df[['ticker', 'number_of_trades', 'win_ratio(%)', 'average_res(%)', 'cumulative_result(%)', 'hold_result(%)',
       'average_trade_len(days)', 'median_res(%)', 'cumulative_result', 'sector', 'industry', 'description','first_data_date', 'first_open_price','last_close_price', 'hold_result',
       'trade_results', 'profitable_trade_results', 'profitable_trades_mean',
       'profitable_trades_median', 'looser_trade_results',
       'looser_trades_mean', 'looser_trades_median', 'median_trade_len(days)',
       'number_of_win_trades', 'number_of_lost_trades', 'max_gain(%)',
       'max_lost(%)', 'first_trade_buy',
       'last_data_date',  'buy_threshold',
       'sell_threshold', 'name'
       ]]

from google.colab import data_table
data_table.DataTable(summary_df, include_index=False, num_rows_per_page=10)

,ticker,number_of_trades,win_ratio(%),average_res(%),cumulative_result(%),hold_result(%),average_trade_len(days),median_res(%),cumulative_result,sector,...,median_trade_len(days),number_of_win_trades,number_of_lost_trades,max_gain(%),max_lost(%),first_trade_buy,last_data_date,buy_threshold,sell_threshold,name
0,V,5,100.00,118.09,2037.0,2082.0,998.0,53.02,21.37 x,Commercial Services,...,878.0,5,0,409.10,6.21,2008-10-08,2025-04-10,30,80,V
1,SPGI,6,66.67,48.53,430.0,647.0,844.0,49.40,5.3 x,Commercial Services,...,829.0,4,2,180.98,-44.83,2007-07-12,2025-04-10,30,80,SPGI
2,MCO,7,71.43,19.98,174.0,601.0,616.0,28.30,2.74 x,Commercial Services,...,466.0,5,2,54.41,-37.07,2007-06-26,2025-04-10,30,80,MCO
3,PYPL,2,50.00,3.90,0.0,59.0,1494.0,3.90,1.0 x,Commercial Services,...,1493.5,1,1,32.70,-24.91,2015-08-25,2025-04-10,30,80,PYPL
4,EFX,8,75.00,40.64,700.0,484.0,639.0,18.64,8.0 x,Commercial Services,...,556.0,6,2,188.81,-14.03,2007-07-27,2025-04-10,30,80,EFX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,EXC,5,80.00,10.86,17.0,-12.0,1201.0,2.99,1.17 x,Utilities,...,1219.0,4,1,68.91,-49.15,2008-01-23,2025-04-10,30,80,EXC
196,D,4,75.00,13.82,45.0,15.0,1506.0,25.43,1.45 x,Utilities,...,1473.5,3,1,36.92,-32.51,2007-05-29,2025-04-10,30,80,D
197,SRE,2,50.00,72.90,130.0,113.0,3209.0,72.90,2.3 x,Utilities,...,3209.0,1,1,155.67,-9.87,2007-06-07,2025-04-10,30,80,SRE
198,PEG,6,83.33,17.01,138.0,86.0,922.0,13.49,2.38 x,Utilities,...,815.5,5,1,52.51,-7.30,2008-03-27,2025-04-10,30,80,PEG


from matplotlib import pyplot as plt
_df_0['number_of_trades'].plot(kind='hist', bins=20, title='number_of_trades')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['win_ratio(%)'].plot(kind='hist', bins=20, title='win_ratio(%)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['average_res(%)'].plot(kind='hist', bins=20, title='average_res(%)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['cumulative_result(%)'].plot(kind='hist', bins=20, title='cumulative_result(%)')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['first_data_date']
  ys = series['cumulative_result(%)']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values('first_data_date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('first_data_date')
_ = plt.ylabel('cumulative_result(%)')

from matplotlib import pyplot as plt
_df_12['number_of_trades'].plot(kind='line', figsize=(8, 4), title='number_of_trades')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['win_ratio(%)'].plot(kind='line', figsize=(8, 4), title='win_ratio(%)')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['average_res(%)'].plot(kind='line', figsize=(8, 4), title='average_res(%)')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15['cumulative_result(%)'].plot(kind='line', figsize=(8, 4), title='cumulative_result(%)')
plt.gca().spines[['top', 'right']].set_visible(False)

In [164]:
# --- 1. Nyerési arány (%) histogram


fig1 = px.histogram(
    df,
    x='win_ratio(%)',
    nbins=100,
    title='Nyerési arány (%) eloszlása',
    labels={'win_ratio(%)': 'Nyerési arány (%)'},
    template='plotly_white'
)
fig1.show()

fig2 = px.scatter(
    df,
    x='win_ratio(%)',
    y='number_of_trades',
    hover_data=['ticker', 'sector', 'industry', 'cumulative_result', 'hold_result'],
    color='sector',
    title='Win Ratio (%) vs Number of Trades',
    labels={
        'win_ratio(%)': 'Win Ratio (%)',
        'number_of_trades': 'Number of Trades'
    },
    template='plotly_white'
)
fig2.show()

# Group by sector and calculate the average and count of tickers
sector_avg = df.groupby('sector', as_index=False).agg(
    average_res=('average_res(%)', 'mean'),
    tickers_count=('ticker', 'nunique')  # Count unique tickers
)

# Sort by the average result
sector_avg.sort_values(by='average_res', ascending=False, inplace=True)
sector_avg.reset_index(drop=True, inplace=True)

# Create the bar plot for sector
fig3 = px.bar(
    sector_avg,
    y='sector',
    x='average_res',
    color='sector',
    title='Average Result by Sector with Ticker Count',
    labels={
        'sector': 'Sector',
        'average_res': 'Average Result (%)',
        'tickers_count': 'Number of Tickers'
    },
    text='tickers_count',  # Add the number of tickers as text labels
    template='plotly_white',
)

# Optional: make the x labels more readable if there are many sectors
fig3.update_traces(textposition='outside')  # Place text outside the bars
fig3.show()

# Group by industry and calculate the average and count of tickers
industry_avg = df.groupby('industry', as_index=False).agg(
    average_res=('average_res(%)', 'mean'),
    tickers_count=('ticker', 'nunique')  # Count unique tickers
)

# Sort by the average result
industry_avg.sort_values(by='average_res', ascending=False, inplace=True)
industry_avg.reset_index(drop=True, inplace=True)

# Create the bar plot for industry
fig4 = px.bar(
    industry_avg,
    y='industry',
    x='average_res',
    color='industry',
    title='Average Result by Industry with Ticker Count',
    labels={
        'industry': 'Industry',
        'average_res': 'Average Result (%)',
        'tickers_count': 'Number of Tickers'
    },
    text='tickers_count',  # Add the number of tickers as text labels
    template='plotly_white',
)

# Optional: make the x labels more readable if there are many industries
fig4.update_traces(textposition='outside')  # Place text outside the bars
fig4.show()


# Buborékdiagram a fő teljesítménymutatókról
bubble_fig = px.scatter(
    df,
    x="win_ratio(%)",
    y="average_res(%)",
    size="number_of_trades",
    color="sector",
    hover_name="ticker",
    size_max=50,
    hover_data={
        "cumulative_result": True,
        "max_gain(%)": True,
        "max_lost(%)": True,
        "number_of_win_trades": True,
        "number_of_lost_trades": True
    },
    labels={
        "win_ratio(%)": "Nyerési arány (%)",
        "average_res(%)": "Átlagos eredmény (%)",
        "number_of_trades": "Kereskedések száma",
        "sector": "Szektor"
    },
    title="Részvények teljesítménye (buborékok mérete = kereskedések száma)"
)

bubble_fig.update_layout(
    template="plotly_white",
    legend=dict(orientation="h", y=-0.15)
)
bubble_fig.show()
